In [1]:
import numpy as np
import torch

print("Torch version:", torch.__version__)

Torch version: 1.7.1


In [2]:
! rm ./clip_classes/.DS_Store

In [3]:
import os
#our the classes and images we want to test are stored in folders in the test set
class_names = os.listdir('./clip_classes/')
class_names.remove('_tokenization.txt')
class_names

['sleeping', 'sitting', 'jogging']

In [4]:
#we auto generate some example tokenizations in Roboflow but you should edit this file to try out your own prompts
#CLIP gets a lot better with the right prompting!
#be sure the tokenizations are in the same order as your class_names above!
%cat ./clip_classes/_tokenization.txt

A picture of a person sleeping
A picture of a person sitting
A picture of a person jogging


In [5]:
%%writefile ./clip_classes/_tokenization.txt
A picture of a person sleeping
A picture of a person sitting
A picture of a person jogging

Overwriting ./clip_classes/_tokenization.txt


In [6]:
candidate_captions = []
with open('./clip_classes/_tokenization.txt') as f:
    candidate_captions = f.read().splitlines()

In [7]:
import torch
import clip
from PIL import Image
import glob

def argmax(iterable):
    return max(enumerate(iterable), key=lambda x: x[1])[0]

device = "cuda" if torch.cuda.is_available() else "cpu"
model, transform = clip.load("ViT-B/32", device=device)

correct = []

#define our target classificaitons, you can should experiment with these strings of text as you see fit, though, make sure they are in the same order as your class names above
text = clip.tokenize(candidate_captions).to(device)

for cls in class_names:
    class_correct = []
    test_imgs = glob.glob('./clip_classes/' + cls + '/*.jpg')
    for img in test_imgs:
        print(img.split('/')[-1])
        image = transform(Image.open(img)).unsqueeze(0).to(device)
        with torch.no_grad():
            image_features = model.encode_image(image)
            text_features = model.encode_text(text)
            
            logits_per_image, logits_per_text = model(image, text)
            probs = logits_per_image.softmax(dim=-1).cpu().numpy()

            pred = class_names[argmax(list(probs)[0])]
            print("Prediction Class : %s,  Label : %d,  Confidence : %.4f" % (pred, argmax(list(probs)[0]), probs[0][argmax(list(probs)[0])]*100))
            if pred == cls:
                correct.append(1)
                class_correct.append(1)
            else:
                correct.append(0)
                class_correct.append(0)
    
    print('\nAccuracy on class ' + cls + ' is : ' + str(sum(class_correct)/len(class_correct)*100) + "%\n\n")
print('\nOverall Accuracy is : ' + str(sum(correct)/len(correct)*100) + "%")

sleeping4.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 90.7468
sleeping5.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 99.9622
sleeping7.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 99.9920
sleeping6.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 99.9726
sleeping1.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 99.9823
sleeping8.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 99.8401
sleeping9.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 99.0630
sleeping10.jpg
Prediction Class : sleeping,  Label : 0,  Confidence : 98.8622

Accuracy on class sleeping is : 100.0%


sitting6.jpg
Prediction Class : sitting,  Label : 1,  Confidence : 99.9006
sitting7.jpg
Prediction Class : sitting,  Label : 1,  Confidence : 99.7139
sitting5.jpg
Prediction Class : sitting,  Label : 1,  Confidence : 97.7814
sitting4.jpg
Prediction Class : sitting,  Label : 1,  Confidence : 99.9648
sitting1.jpg
Prediction Class : sitting, 